In [5]:
from functools import wraps
from IPython.display import display, HTML, Markdown, display
from ipywidgets import *
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
import seaborn as sns
from MCForecastTools import MCSimulation
import openai
import warnings
warnings.filterwarnings("ignore", message="DataFrame is highly fragmented.")


%matplotlib inline

In [6]:
load_dotenv()

True

In [10]:
# define the stocks
stocks = ['JPM', 'JNJ', 'MSFT', 'RIO', 'PFE']

# define the widgets for user input
stock_dropdown = Dropdown(options=stocks, description='Select a stock:', style={'description_width': 'initial', 'width': 'auto'})
weight_input = FloatText(description='Weight:', value=0.0)

# define the output widget for the summary, graphs
summary_output = Output()
daily_return_output = Output()
cumulative_return_output = Output()
std_analysis_output = Output()
rolling_std_analysis_output = Output()
correlation_output = Output()
beta_output = Output()
beta_graph_output = Output()
sharpe_ratio_output = Output()
monte_carlo_output = Output()

# define the labels
selection_label = Label()
selected_stocks_label = Label(value='Selected stocks: ')
selected_weights_label = Label(value='Selected weights: ')

# define the function
selected_stocks = []
selected_weights = []

# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
# Create the Alpaca API object
api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)
# Set up OpenAI API key
openai.api_key = os.getenv("OPENAI_API_key")


def select_stock_and_weight(button):
    global selected_stocks, selected_weights
    # check max stocks are selected
    if len(selected_stocks) >= 3:
        selection_label.value = "Maximum of 3 stocks allowed"
        return
    # add the current selection to the list of stocks and weights
    selected_stock = stock_dropdown.value
    if selected_stock in selected_stocks:
        selection_label.value = "Cannot have duplicated stocks"
        return
    selected_weight = weight_input.value
    selected_stocks.append(selected_stock)
    selected_weights.append(selected_weight)
    # update the selection label
    selected_stocks_label.value += f'{selected_stock}, '
    selected_weights_label.value += f'{selected_weight}, '
    

def submit(button):
    global selected_stocks, selected_weights, stock_name1, stock_name2, stock_name3, stock1_weight, stock2_weight, stock3_weight, weights, initial_investment, daily_returns, stock1_std, stock2_std, stock3_std, spy_annualised_std, portfolio_annualised_std
    # check that the sum of weights is equal to 1
    total_weight = sum(selected_weights)
    if total_weight != 1:
        selection_label.value = "Weights must add up to 1"
        return
    # check max stocks are selected
    if len(selected_stocks) > 3:
        selection_label.value = "Please select Maximum three stocks"
        return
    # get the initial investment value
    initial_investment = initial_investment_input.value
    # create variables to hold the selected stocks, weights, and initial investment
    analysis_stocks = selected_stocks.copy()
    analysis_weights = selected_weights.copy()
    
    # get stock names and weights for summary
    stock_name1 = analysis_stocks[0]
    stock_name2 = analysis_stocks[1]
    stock_name3 = analysis_stocks[2]
    stock1_weight = float(analysis_weights[0])
    stock2_weight = float(analysis_weights[1])
    stock3_weight = float(analysis_weights[2])
    weights = [stock1_weight, stock2_weight, stock3_weight]
    initial_investment = initial_investment_input.value
    
    
    # generate the summary using OpenAI API
    prompt = f"Please briefly summarize what is the main products or service and full names of {stock_name1}, {stock_name2} and {stock_name3} and what they are working on in separate small paragraphs"
    model = "text-davinci-002"
    response = openai.Completion.create(
        engine=model,
        prompt=prompt,
        max_tokens=1024,
        n=1,
        stop=None,
        temperature=0.5,
    )
    company_info = response.choices[0].text.strip()
    paragraphs = company_info.split('\n\n\n')    
    # update the labels with the summary
    selected_stocks_label.value = 'Selected stocks: ' + ', '.join(analysis_stocks)
    selected_weights_label.value = 'Selected weights: ' + ', '.join([str(w) for w in analysis_weights])
    selection_label.value = "Selection submitted"
    
    # display the summary in the output widget
    with summary_output:
        display(Markdown(f"### Brief summary of the selected stocks:\n"))
        for i, paragraph in enumerate(paragraphs):
            display(Markdown(f"#### \n{paragraph}\n"))
            
    
    # get the DataFrame for the selected stocks and SPY
    start_date = pd.Timestamp("2018-05-07", tz="America/New_York").isoformat()
    end_date = pd.Timestamp("2023-05-07", tz="America/New_York").isoformat()
    tickers = [stock_name1, stock_name2, stock_name3, 'SPY']
    timeframe = "1Day"
    df_portfolio = api.get_bars(
        tickers,
        timeframe,
        start=start_date,
        end=end_date
    ).df
    SPY = df_portfolio[df_portfolio['symbol']=='SPY'].drop('symbol', axis=1)
    stock1 = df_portfolio[df_portfolio['symbol']== stock_name1].drop('symbol', axis=1)
    stock2 = df_portfolio[df_portfolio['symbol']== stock_name2].drop('symbol', axis=1)
    stock3 = df_portfolio[df_portfolio['symbol']== stock_name3].drop('symbol', axis=1)
    # Concatenate the ticker DataFrames
    df_stock_data = pd.concat([stock1['close'], stock2['close'], stock3['close'], SPY['close']], axis=1, keys=[stock_name1, stock_name2, stock_name3, 'SPY'])
    # Calculate daily returns for all portfolios
    daily_returns = df_stock_data.pct_change().dropna().copy()
    # Calculate portfolio return and update Daily Returns DF
    stock_daily_returns = daily_returns.drop('SPY', axis=1)
    portfolio_return = stock_daily_returns.dot(weights)
    daily_returns["Custom Portfolio"] = portfolio_return
    # Calculate cumulative returns
    cumulative_returns = (1 + daily_returns).cumprod() - 1
    # Calculate the rolling standard deviation for all portfolios using a 21-day window
    window_size = 21
    rolling_std = daily_returns.rolling(window_size).std()
    # Calculate correlation
    correlation = daily_returns.corr()
    
    
    # Visualize the distribution of daily returns across all stocks using a histogram plot
    fig, axs = plt.subplots(3, 1, figsize=(10, 12))
    for stock_name in daily_returns.columns:
        axs[0].hist(daily_returns[stock_name], bins=50, alpha=0.5, label=stock_name)
    axs[0].set_title('Daily Returns Histogram', fontsize=14)
    axs[0].set_xlabel('Daily Returns', fontsize=12)
    axs[0].set_ylabel('Frequency', fontsize=12)
    axs[0].legend(fontsize=12)
    # Create box plot of daily returns for all portfolios
    sns.boxplot(data=daily_returns, palette="Set2", ax=axs[1])
    axs[1].set_title('Daily Returns Box Plot', fontsize=14)
    axs[1].set_xlabel('Portfolio', fontsize=12)
    axs[1].set_ylabel('Daily Returns', fontsize=12)
    axs[1].set(ylim=(-0.15, 0.15))
    # Show daily returns for all portfolios
    daily_returns.plot(figsize=(14, 16), ax=axs[2], title='Daily Returns', fontsize=14)
    axs[2].set_xlabel('Date', fontsize=12)
    axs[2].set_ylabel('Daily Returns', fontsize=12)
    axs[2].legend(fontsize=12)    
    plt.subplots_adjust(hspace=1)
    plt.tight_layout()    
    # display the graphs in the output widget
    with daily_return_output:
        display(Markdown(f"### Daily Return Graphs:\n"))
        plt.show()
        

    # plot cumulative returns
    fig, ax = plt.subplots(2, 1, figsize=(8, 10))
    ax[0].plot(cumulative_returns.iloc[:, 0], color='blue')
    ax[0].fill_between(cumulative_returns.index, cumulative_returns.iloc[:, 0], 1,
                    where=cumulative_returns.iloc[:, 0] >= 1, interpolate=True,
                    color='green', alpha=0.5)
    ax[0].fill_between(cumulative_returns.index, cumulative_returns.iloc[:, 0], 1,
                    where=cumulative_returns.iloc[:, 0] <= 1, interpolate=True,
                    color='red', alpha=0.5)
    ax[0].set_title('Cumulative Returns')
    ax[0].set_xlabel('Date')
    ax[0].set_ylabel('Cumulative Returns')
    # normal cumulative plot
    cumulative_returns.plot(figsize=(8, 10), title="Cumulative Returns", ax=ax[1])
    ax[1].legend(fontsize=12)    
    plt.subplots_adjust(hspace=1)
    plt.tight_layout()    
    # display the cumulative_return graphs in the output widget
    with cumulative_return_output:
        display(Markdown(f"### Cumulative Return Graphs:\n"))
        plt.show()    
        
    
    
    # Calculate the daily standard deviations of all portfolios
    daily_std = daily_returns.std()
    stock1_std = daily_returns[stock_name1].std()
    stock2_std = daily_returns[stock_name2].std()
    stock3_std = daily_returns[stock_name3].std()
    spy_stock = daily_returns['SPY'].std()
    portfolio_std = daily_returns['Custom Portfolio'].std()
    # Calculate the annualised standard deviation 
    annualised_std = daily_std * np.sqrt(252)
    stock1_annualised_std = stock1_std * np.sqrt(252)
    stock2_annualised_std = stock2_std * np.sqrt(252)
    stock3_annualised_std = stock2_std * np.sqrt(252)
    spy_annualised_std = spy_stock * np.sqrt(252)
    portfolio_annualised_std = portfolio_std * np.sqrt(252)
   # Check which stocks are riskier than S&P 500
    riskier_stocks = []
    for stock in daily_returns.columns:
        if stock == 'SPY' or stock == 'Custom Portfolio':
            continue
        stock_annualised_std = daily_returns[stock].std() * np.sqrt(252)
        if stock_annualised_std > spy_annualised_std:
            riskier_stocks.append(stock)

    std_analysis_output.clear_output()
    with std_analysis_output:
        display(Markdown(f"### Standard Deviation Analysis:\n"))
        print(annualised_std)
        if len(riskier_stocks) == 0:
            display(Markdown("#### Based on the value of annualised standard deviations, all three stocks are less risky than the S&P 500 index."))
        elif len(riskier_stocks) == 1:
            display(Markdown(f"#### Based on the value of annualised standard deviations, {riskier_stocks[0]} is riskier than the S&P 500 index."))
        else:
            display(Markdown(f"#### Based on the value of annualised standard deviations, {', '.join(riskier_stocks)} are riskier than the S&P 500 index."))

        if portfolio_annualised_std > spy_annualised_std:
            display(Markdown("#### The custom portfolio is riskier than the market (SPY)."))
        else:
            display(Markdown("#### The custom portfolio is less risky than the market (SPY)."))
            
            
        #Plot rolling std 21-day
        fig, ax = plt.subplots(figsize=(10, 5))
        # Set title, x-label, y-label, and grid
        ax.set_title('Rolling 21-Day Standard Deviation')
        ax.set_xlabel('Date')
        ax.set_ylabel('Standard Deviation')
        ax.grid(True)
        # Plot the rolling standard deviation
        rolling_std.plot(ax=ax)
        # Add legend outside of the plot
        ax.legend(loc='upper left', bbox_to_anchor=(1, 1), fontsize=10)
        plt.tight_layout()
 
    with rolling_std_analysis_output:  
        display(Markdown(f"### Rolling Standard Deviation Graph:\n"))
        plt.show()
        
        
        # Plot correlation matrix using heatmap
        colors = sns.color_palette("PuBuGn", n_colors=100)
        sns.set(font_scale=1.2)
        plt.figure(figsize=(10, 8))
        sns.heatmap(correlation, annot=True, cmap=colors)
        plt.title("Correlation Heatmap")
        plt.xlabel('Portfolio')
        plt.ylabel('Portfolio')
        plt.tight_layout()
    correlation_output.clear_output(wait=True)
    with correlation_output:
        display(Markdown(f"### Correlation Heatmap:\n"))
        plt.show()
        
        
        # Calculate beta
        covariance_matrix = daily_returns.cov()
        stock1_beta = covariance_matrix.loc[stock_name1]['SPY'] / covariance_matrix.loc['SPY']['SPY']
        stock2_beta = covariance_matrix.loc[stock_name2]['SPY'] / covariance_matrix.loc['SPY']['SPY']
        stock3_beta = covariance_matrix.loc[stock_name3]['SPY'] / covariance_matrix.loc['SPY']['SPY']
        portfolio_beta = covariance_matrix.loc['Custom Portfolio']['SPY'] / covariance_matrix.loc['SPY']['SPY']
        spy_beta = covariance_matrix.loc['SPY']['SPY'] / covariance_matrix.loc['SPY']['SPY']
        # Display beta values
    with beta_output:
        display(Markdown(f"#### {stock_name1} beta: {stock1_beta:.2f}"))
        if stock1_beta > spy_beta:
            display(Markdown(f"#### {stock_name1} is more volatile than the overall market."))
        elif stock1_beta < spy_beta:
            display(Markdown(f"#### {stock_name1} is less volatile than the overall market."))
        else:
            display(Markdown(f"#### {stock_name1} has the same volatility as the overall market."))
        display(Markdown(f"#### {stock_name2} beta: {stock2_beta:.2f}"))
        if stock2_beta > spy_beta:
            display(Markdown(f"#### {stock_name2} is more volatile than the overall market."))
        elif stock2_beta < spy_beta:
            display(Markdown(f"#### {stock_name2} is less volatile than the overall market."))
        else:
            display(Markdown(f"#### {stock_name2} has the same volatility as the overall market."))
        display(Markdown(f"#### {stock_name3} beta: {stock3_beta:.2f}"))
        if stock3_beta > spy_beta:
            display(Markdown(f"#### {stock_name3} is more volatile than the overall market."))
        elif stock3_beta < spy_beta:
            display(Markdown(f"#### {stock_name3} is less volatile than the overall market."))
        else:
            display(Markdown(f"#### {stock_name3} has the same volatility as the overall market."))
        display(Markdown(f"#### Custom Portfolio beta: {portfolio_beta:.2f}"))
        if portfolio_beta > spy_beta:
            display(Markdown(f"#### The custom portfolio is more volatile than the overall market."))
        elif portfolio_beta < spy_beta:
            display(Markdown(f"#### The custom portfolio is less volatile than the overall market."))
        else:
            display(Markdown(f"#### The custom portfolio has the same volatility as the overall market."))

        # Display beta values graph
    with beta_graph_output:
        display(Markdown(f"### Beta Value Graph:\n"))
        beta_values = [stock1_beta, stock2_beta, stock3_beta, portfolio_beta]
        company_names = [stock_name1, stock_name2, stock_name3, 'Custom Portfolio']
        colors = ['#ADD8E6']
        plt.bar(company_names, beta_values, color=colors)
        plt.title('Beta Values for Selected Companies')
        plt.xlabel('Portfolio')
        plt.ylabel('Beta Value')
        plt.gca().set_facecolor('#6B7C96')
        plt.grid(False)
        plt.show()    
    
        
        # Sharpe Ratio
        sharpe_ratio = (daily_returns.mean() * 252)/ (daily_returns.std() * np.sqrt(252))
        # Convert the Sharpe ratio output to a string
        sharpe_ratio_str = sharpe_ratio.to_string()
        # Generate a summary using GPT-3
        prompt_sharpe = f"Please write a simple conclusion of the custom portfolio based on its Sharpe ratio. Provide a brief summary. \n\n{sharpe_ratio_str}"
        response = openai.Completion.create(
            engine="text-davinci-002",
            prompt=prompt_sharpe,
            max_tokens=100,
            n=1,
            stop=None,
            temperature=0.5,
        )
        # Print the summary generated by GPT-3
        sharpe_ratio_analysis = response.choices[0].text.strip()
        # display the summary in the output widget
    with sharpe_ratio_output:
        display(Markdown(f"### Brief analysis on Sharpe Ratio:\n"))
        display(sharpe_ratio_analysis)
        
        
        
        #Monte Carlo Simulation
        df_stock_combined = pd.concat([stock1,stock2,stock3], axis=1, keys=[stock_name1,stock_name2,stock_name3])
        MC_ten_year = MCSimulation(
            portfolio_data = df_stock_combined,
            weights = weights,
            num_simulation = 500,
            num_trading_days = 252 * 10
        )
        
    with monte_carlo_output:
        # Run a Monte Carlo simulation to forecast ten years cumulative returns
        MC_ten_year.calc_cumulative_return()
        simulated_returns_data = {
            "mean": list(MC_ten_year.simulated_return.mean(axis=1)),
            "median": list(MC_ten_year.simulated_return.median(axis=1)),
            "min": list(MC_ten_year.simulated_return.min(axis=1)),
            "max": list(MC_ten_year.simulated_return.max(axis=1))
        }
        # Create a DataFrame with the summary statistics
        df_simulated_returns = pd.DataFrame(simulated_returns_data)
        # Multiply an initial investment by the daily returns of simulative stock prices to return the progression of daily returns in terms of money
        cumulative_pnl = initial_investment_input.value * df_simulated_returns
        # Fetch summary statistics from the Monte Carlo simulation results
        tbl = MC_ten_year.summarize_cumulative_return()       
        ci_lower = round(tbl[8]*initial_investment,2)
        ci_upper = round(tbl[9]*initial_investment,2)
        str_initial = str(initial_investment)
        display(Markdown(f"### 500 Monte Carlo Simulation over the next 10 years:\n"))
        display(Markdown(f"#### There is a 95% chance that an initial investment of ${str_initial} in the portfolio over the next year will end within the range of ${ci_lower:.2f} and ${ci_upper:.2f}."))
        
        
        
        
    # create container widgets for each output
    daily_return_container = VBox([daily_return_output])
    cumulative_return_container = VBox([cumulative_return_output])
    std_analysis = VBox([std_analysis_output])
    rolling_std_analysis = VBox([rolling_std_analysis_output])
    correlation = VBox([correlation_output]) 
    beta = VBox([beta_output])
    beta_graph = VBox([beta_graph_output])
    sharpe_ratio = VBox([sharpe_ratio_output])
    monte_carlo = VBox([monte_carlo_output])
    inputs.children += (summary_output, daily_return_container, cumulative_return_container, std_analysis, rolling_std_analysis, correlation,
                       beta, beta_graph, sharpe_ratio, monte_carlo)
    
    
    
    
def clear(button):
    global selected_stocks, selected_weights
    selected_stocks = []
    selected_weights = []
    selection_label.value = ""
    selected_stocks_label.value = 'Selected stocks: '
    selected_weights_label.value = 'Selected weights: '
    stock_dropdown.value = stocks[0]
    weight_input.value = 0.0
    initial_investment_input.value = 0.0
    

# select and submit buttons
select_button = Button(description='Select')
select_button.on_click(select_stock_and_weight)
submit_button = Button(description='Submit')
submit_button.on_click(submit)
clear_button = Button(description='Clear')
clear_button.on_click(clear)


# create initial investment widgets
initial_investment_label = Label(value='Enter initial investment:')
initial_investment_input = FloatText(value=0.0)

inputs = VBox([stock_dropdown, weight_input, select_button, selected_stocks_label, selected_weights_label, selection_label, initial_investment_label, initial_investment_input, submit_button, clear_button])
display(inputs)


inputs.layout.border = 'solid 8px'
inputs.layout.padding = '10px'
inputs.layout.margin = '10px'
inputs.layout.width = '100%'
#Add style to the summary and output widgets
summary_output.add_class("output_style")
summary_output.layout = Layout(border="1px solid black", padding="10px", margin="10px")
std_analysis_output.add_class("output_style")
std_analysis_output.layout = Layout(border="1px solid black", padding="10px", margin="10px")
beta_output.add_class("output_style")
beta_output.layout = Layout(border="1px solid black", padding="10px", margin="10px")
sharpe_ratio_output.add_class("output_style")
sharpe_ratio_output.layout = Layout(border="1px solid black", padding="10px", margin="10px")
monte_carlo_output.add_class("output_style")
monte_carlo_output.layout = Layout(border="1px solid black", padding="10px", margin="10px")
#Add style to the plot and output widgets
for output in [daily_return_output, cumulative_return_output]:
    output.add_class("output_style")
    output.layout = Layout(width="150%", height="600px")


In [ ]:
# Visualize the sharpe ratio vs Daily Standard Deviation
tickers = [stock_name1, stock_name2, stock_name3, 'SPY', 'Custom Portfolio']
sharpe_ratios = [sharpe_ratio[0], sharpe_ratio[1], sharpe_ratio[2], sharpe_ratio[3], sharpe_ratio[4]]
daily_std = [stock1_std, stock2_std, stock3_std, spy_stock, portfolio_std]
# Scatter plot with Sharpe ratios as color chart
fig, ax = plt.subplots(figsize=(10, 8))
scatter = ax.scatter(daily_std, sharpe_ratios, c=sharpe_ratios, cmap='Blues_r', s=200)
ax.set_xlabel('Daily Standard Deviation')
ax.set_ylabel('Sharpe Ratio')
ax.set_title('Sharpe Ratio vs Daily Standard Deviation')
ax.set_facecolor('#F4C2C2')
# Add color bar
cbar = fig.colorbar(scatter)
cbar.set_label
# Add ticker labels to points
for i, ticker in enumerate(tickers):
    ax.annotate(ticker, (daily_std[i], sharpe_ratios[i]), textcoords="offset points", xytext=(0,10), ha='center')
plt.show()